<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
_ Chosen features fed into the system. _
### Hidden Layer:
_ Better thought of as an un-monitored layer, containes defined nodes that will adjust within given parameters _
### Output Layer:
_ Where the system provides its calculated value _
### Neuron:
_ One node within the system _
### Weight:
_ How much significance is assigned to a node _
### Activation Function:
_ What causes a node to fire _
### Node Map:
_ Visual representation of the network _
### Perceptron:
_ Entry level Neural Network that has been around since the 60s in simple forms _

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
import numpy as np

np.random.seed(734)

In [2]:
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [3]:
df.head()

,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [4]:
inputs = np.array(df)
print(inputs.shape)
inputs

(4, 3)


array([[0, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 0]])

In [5]:
target_outputs = []
for i in df['y']:
    target_outputs.append([i])
print('target out:', target_outputs)

target out: [[1], [1], [1], [0]]


In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [7]:
weights = 2 * np.random.random((len(inputs[0]),1)) - 1
weights

array([[ 0.58097096],
       [-0.36145724],
       [-0.26251945]])

In [8]:
weighted_sum = np.dot(inputs, weights)
weighted_sum

array([[-0.26251945],
       [ 0.3184515 ],
       [-0.6239767 ],
       [ 0.21951371]])

In [9]:
activated_output = sigmoid(weighted_sum)
print(f'target output:{target_outputs}\n')
print(f'activated output:{activated_output}')

target output:[[1], [1], [1], [0]]

activated output:[[0.43474447]
 [0.57894683]
 [0.34887755]
 [0.55465912]]


In [10]:
error = target_outputs - activated_output
error

array([[ 0.56525553],
       [ 0.42105317],
       [ 0.65112245],
       [-0.55465912]])

In [11]:
adjusted = error * sigmoid_derivative(activated_output)
adjusted

array([[ 0.13484154],
       [ 0.0969131 ],
       [ 0.15792615],
       [-0.12852376]])

In [12]:
weights += np.dot(inputs.T, adjusted)
weights

array([[ 0.54936029],
       [-0.33205485],
       [ 0.12716134]])

In [13]:
for iteration in range(10000):
    
    # Weighted sum of inputs / weights
    weighted_sum = np.dot(inputs, weights)
    
    # Activate!
    activated_output = sigmoid(weighted_sum)
    
    # Calc error
    error = target_outputs - activated_output
    
    adjusted = error * sigmoid_derivative(activated_output)
    
    # Update the Weights
    weights += np.dot(inputs.T, adjusted)
    
print("\nWeights after training")
print(weights)

print("\nOutput after training")
print(activated_output)

print("\nOutput after training (rounded)")
print(activated_output.round())


Weights after training
[[-3.80352236]
 [-3.81018591]
 [11.6966912 ]]

Output after training
[[9.99991678e-01]
 [9.99626818e-01]
 [9.99624323e-01]
 [4.93443411e-04]]

Output after training (rounded)
[[1.]
 [1.]
 [1.]
 [0.]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [14]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [15]:
type(diabetes['Outcome'])

pandas.core.series.Series

In [16]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

scaler = MinMaxScaler()
X = diabetes[feats]
y = diabetes[['Outcome']]

scaled_df = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
scaled_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000


In [18]:
X = scaled_df[feats]
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000


In [ ]:
class Perceptron(object):
    
    def __init__(self, niter = 10):
    self.niter = niter
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        sx = sigmoid(x)
        return sx * (1-sx)

    def fit(self, X, y):
        """
        Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """

        # Randomly Initialize Weights
        self.weights = ...

    for i in range(self.niter):
        # Weighted sum of inputs / weights

        # Activate!

        # Calc error

        # Update the Weights


    def predict(self, X):
    """Return class label after unit step"""
        return None

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?